In [ ]:
pip install transformers

     |████████████████████████████████| 778kB 4.6MB/s 
     |████████████████████████████████| 3.0MB 21.0MB/s 
     |████████████████████████████████| 890kB 43.0MB/s 
     |████████████████████████████████| 1.1MB 44.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=bef625647df1327e271ffe7492498ebcae60e17818476da8bba70e31d06a797b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch

import pandas as pd
import random
import numpy as np
from torchtext import data
from torchtext import datasets
import torch.nn as nn
import spacy
from collections import Counter

from transformers import BertTokenizer, BertModel

import torch.optim as optim

import time
import random

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


In [ ]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [ ]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)

512


In [ ]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [ ]:
tokenize_and_cut('positive and good')

['positive', 'and', 'good']

In [ ]:
tokenizer.convert_tokens_to_ids('.')

1012

In [ ]:
TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

SELECTED_TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

In [ ]:
fields = [(None, None), ('text', TEXT), ('selected_text', SELECTED_TEXT), ('label', LABEL)]
#add selected text!

In [ ]:
train_data = data.TabularDataset.splits(
                                        path = '',
                                        train = 'train_full.csv',
                                        format = 'csv',
                                        fields = fields,
                                        skip_header = True
)

In [ ]:
train_data, test_data = train_data[0].split(random_state = random.seed(SEED), split_ratio=0.8)
train_data, valid_data = train_data.split(random_state = random.seed(SEED), split_ratio=0.8)

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17588
Number of validation examples: 4397
Number of testing examples: 5496


In [ ]:
LABEL.build_vocab(train_data)

In [ ]:
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in LABEL vocabulary: 3


In [ ]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7fc687dc6e18>, {'neutral': 0, 'positive': 1, 'negative': 2})


In [ ]:
BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device, sort=False)

In [ ]:
bert = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]

         
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])

        
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)   
        #output = [batch size, out dim]
        
        return output

In [ ]:
HIDDEN_DIM = 256
OUTPUT_DIM = 2
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 112,241,922 trainable parameters


In [ ]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,759,682 trainable parameters


In [ ]:
for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
out.weight
out.bias


In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
#criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def categorical_accuracy(preds, y):
    # preds is output (3 values)
    # y is other vector of right labels
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y) # return a vector, 1 if the two cells are equal
    return correct.sum() / torch.FloatTensor([y.shape[0]])

In [ ]:
def intersection(lst1, lst2): 
    #return [value for value in lst1 if value in lst2] 
    return list((Counter(lst1.tolist()) & Counter(lst2.tolist())).elements()) 

In [ ]:
def jaccard_similarity(list1,list2):
    lenIntersection=len(intersection(list1,list2))
    union=(len(list1)+len(list2)) - lenIntersection
    if union == 0:
        return 0.0
    else:
        return float(lenIntersection)/union

In [ ]:
#calculating jaccard for pytorch tensors
def jaccard(text, pred, y):
    jaccard = 0
    for i, v in enumerate(text):
        jacc_value = jaccard_similarity(v[pred[i][0].round().int().item():pred[i][1].round().int().item()],
                                      v[y[i][0].round().int().item():y[i][1].round().int().item()])
        jaccard += jacc_value
    result = jaccard / torch.FloatTensor([text.shape[0]])
    return result

In [ ]:
#converting labels into bert ids
def LabelToId(x):
    return tokenizer.convert_tokens_to_ids(LABEL.vocab.itos[x])

In [ ]:
#init token + label id + pad token + pad token + text ids
def process(text, label):
    result = torch.tensor(np.full((label.shape[0],1), init_token_idx), dtype=torch.int64, device=device)
    pad = torch.tensor(np.full((label.shape[0],1), pad_token_idx), dtype=torch.int64, device=device)
    labels_idx = torch.empty((label.shape), dtype=torch.int64, device=device)
    for i, v in enumerate(label):
        labels_idx[i] = LabelToId(int(v.item()))
    labels_idx = labels_idx.unsqueeze(1)
    result = torch.cat((result, labels_idx), 1)
    result = torch.cat((result, pad) , 1)
    result = torch.cat((result, pad) , 1)

    result = torch.cat((result, text[:, 1:]), 1)

    return result

In [ ]:
#getting begin position and end position of selected text in original text
def get_positions(text, selected_text):
    selected_text_ = selected_text[:, 1:]
    text_ = text[:, 1:]
    result = torch.empty((1,2), dtype=torch.int64)
    for i, v in enumerate(text_):
        start_position = 0
        end_position = 0
        for j in range(v.shape[0]):
            if selected_text_[i][0].item() == v[j].item():
                for k in range(selected_text_[i].shape[0]):
                    if selected_text_[i][k].item() == v[j+k].item():
                        if k==0:
                            start_position = j
                        if selected_text_[i][k].item() == eos_token_idx:
                            end_position = j+k
                            break
                    else:
                        end_position = j+k
                        break
        start_position += 4
        end_position += 4
        positions = torch.tensor([[start_position, end_position]], dtype=torch.int64)
        result = torch.cat((result, positions), 0)
    return result[1:,:]

In [ ]:
#creating random positions as baseline
def create_baseline(text, pos):
    result = torch.empty((pos.shape))
    for i, v in enumerate(text):
        first = random.randint(4, (v==eos_token_idx).nonzero().item())
        second = random.randint(first, (v==eos_token_idx).nonzero().item())
        result[i] = torch.tensor([[first, second]])
    return result

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_jacc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        input = process(batch.text, batch.label)

        positions = get_positions(batch.text, batch.selected_text).float().to(device)

        predictions = model(input)
        
        loss = criterion(predictions, positions)
        
        jacc = jaccard(input, predictions, positions)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_jacc += jacc.item()
        
    return epoch_loss / len(iterator), epoch_jacc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_jacc = 0
    epoch_jacc_baseline = 0
    epoch_loss_baseline = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            input = process(batch.text, batch.label)
            positions = get_positions(batch.text, batch.selected_text).float().to(device)

            predictions = model(input)
            
            loss = criterion(predictions, positions)
            
            jacc = jaccard(input, predictions, positions)

            baseline = create_baseline(input, positions).to(device)
            jacc_baseline = jaccard(input, baseline, positions)
            loss_baseline = criterion(baseline, positions)

            epoch_loss += loss.item()
            epoch_jacc += jacc.item()
            epoch_jacc_baseline += jacc_baseline.item()
            epoch_loss_baseline += loss_baseline.item()
        
    return epoch_loss / len(iterator), epoch_jacc / len(iterator), epoch_loss_baseline / len(iterator), epoch_jacc_baseline / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
train_loss_plot = []
train_jacc_plot = []
valid_loss_plot = []
valid_jacc_plot = []

In [ ]:
N_EPOCHS = 20

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_jacc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_jacc, baseline_loss, baseline_jacc = evaluate(model, valid_iterator, criterion)
    train_loss_plot.append(train_loss)
    train_jacc_plot.append(train_jacc)
    valid_loss_plot.append(valid_loss)
    valid_jacc_plot.append(valid_jacc)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'bert-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Jacc: {train_jacc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Jacc: {valid_jacc*100:.2f}%')
    print(f'\t  BL. Loss: {baseline_loss:.3f} |   BL. Jacc: {baseline_jacc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 30s
	Train Loss: 57.400 | Train Jacc: 28.71%
	 Val. Loss: 40.146 |  Val. Jacc: 35.21%
	  BL. Loss: 91.472 |   BL. Jacc: 16.10%
Epoch: 02 | Epoch Time: 1m 29s
	Train Loss: 38.638 | Train Jacc: 36.79%
	 Val. Loss: 32.869 |  Val. Jacc: 41.05%
	  BL. Loss: 90.655 |   BL. Jacc: 16.62%
Epoch: 03 | Epoch Time: 1m 29s
	Train Loss: 35.774 | Train Jacc: 38.52%
	 Val. Loss: 34.173 |  Val. Jacc: 41.19%
	  BL. Loss: 91.529 |   BL. Jacc: 16.26%
Epoch: 04 | Epoch Time: 1m 29s
	Train Loss: 33.468 | Train Jacc: 39.81%
	 Val. Loss: 31.770 |  Val. Jacc: 41.76%
	  BL. Loss: 88.748 |   BL. Jacc: 15.97%
Epoch: 05 | Epoch Time: 1m 29s
	Train Loss: 31.322 | Train Jacc: 40.40%
	 Val. Loss: 31.048 |  Val. Jacc: 43.52%
	  BL. Loss: 94.030 |   BL. Jacc: 15.71%
Epoch: 06 | Epoch Time: 1m 29s
	Train Loss: 29.464 | Train Jacc: 41.15%
	 Val. Loss: 30.740 |  Val. Jacc: 44.11%
	  BL. Loss: 90.930 |   BL. Jacc: 16.14%
Epoch: 07 | Epoch Time: 1m 29s
	Train Loss: 27.756 | Train Jacc: 41.54%
	 Va

In [ ]:
model.load_state_dict(torch.load('bert-model.pt'))

test_loss, test_acc, _ , _ = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Jacc: {test_acc*100:.2f}%')

Test Loss: 32.544 | Test Jacc: 45.06%


In [ ]:
def predict_sentiment(model, tokenizer, sentence, label):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + [tokenizer.convert_tokens_to_ids(label)] +  [pad_token_idx] + [pad_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = model(tensor)
    #max_preds = prediction.argmax(dim = 1)
    return prediction, indexed
    #return max_preds.item()

In [ ]:
sentence = "I am very sorry and today is very ugly."
label = "negative"
pred_class, indexed = predict_sentiment(model, tokenizer, sentence , label)
result = [tokenizer.convert_ids_to_tokens(i) for i in indexed[pred_class[0][0].round().int().item():pred_class[0][1].round().int().item()]]
print(result)

['sorry', 'and', 'today', 'is', 'very']
